In [1]:
INPUT_MARKS  = ["H3K27AC", "H3K27ME3", "H3K36ME3", "H3K4ME1", "H3K4ME3", "INPUT"]
OUTPUT_MARKS = ["H3K27AC"]
MARK_INDEX   = [0, 1, 2, 3, 4, 5]
PEAK_MARK_INDEX = [0, 1, 2, 3, 4]
SEQ_LENGTH = 1001
#DATA_PATH = './data/GM12878_5+1marks-K4me3_all_subsample-0.5e6-None_rS-0_numEx-1000_seqLen-1001_peakFrac-0.5_peaksFac-H3K27AC_norm-arcsinh.npz'
DATA_PATH = 'GM12878.npz'
EVAL_PATH = 'GM18526.npz'
zero_out_non_bins = True

In [2]:
import os 
import numpy as np

def input_not_before_end(list_of_marks):
    return ('INPUT' not in list_of_marks[:-1])

def load_seq_dataset():
    seq_length = SEQ_LENGTH
    input_marks = INPUT_MARKS
    output_marks = OUTPUT_MARKS
  
    assert(input_not_before_end(output_marks))
    assert(input_not_before_end(input_marks))
    
    dataset_path = os.path.join(DATA_PATH)

    #try:      
    with np.load(dataset_path) as data:
        X = data['X'].astype('float32')
        Y = data['Y'].astype('float32')
        peakPValueX = data['peakPValueX'].astype('float32')
        peakPValueY = data['peakPValueY'].astype('float32')
        peakBinaryX = data['peakBinaryX'].astype('int8')
        peakBinaryY = data['peakBinaryY'].astype('int8')
    #except:
        #raise Exception, "Dataset doesn't exist or is missing a required matrix."

    
    marks_idx =  MARK_INDEX
    peak_marks_idx = PEAK_MARK_INDEX
    
    X = X[..., marks_idx]
    peakPValueX = peakPValueX[..., peak_marks_idx]
    peakBinaryX = peakBinaryX[..., peak_marks_idx]

    assert(np.all(peakPValueX >= 0) & np.all(peakPValueY >= 0))

    if (X.shape[0], X.shape[1]) != (Y.shape[0], Y.shape[1]):
        raise Exception("First two dimensions of X and Y shapes (num_examples, seq_length) \
                          need to agree.")
    if (peakPValueX.shape[0], peakPValueX.shape[1]) != (peakPValueY.shape[0], peakPValueY.shape[1]):
        raise Exception("First two dimensions of peakPValueX and peakPValueY shapes \
                          (num_examples, seq_length) need to agree.")
    if len(peakPValueX) != len(X):
        raise Exception("peakPValueX and X must have same length.")

    if ((seq_length != X.shape[1]) or (seq_length != peakPValueX.shape[1])):
        raise Exception("seq_length between model and data needs to agree")

    return X, Y, peakPValueX, peakPValueY, peakBinaryX, peakBinaryY

In [3]:
X, Y, peakPValueX, peakPValueY, peakBinaryX, peakBinaryY = load_seq_dataset()

if zero_out_non_bins:
    peakPValueX = peakPValueX * peakBinaryX
    peakPValueY = peakPValueY * peakBinaryY 

def sq2p_process_X(X):
    return X

def sq2p_process_Y(Y):
    mid = (SEQ_LENGTH - 1) / 2
    mid = int(mid)
    return Y[:, mid:mid+1, :]    
    
X = sq2p_process_X(X)
Y = sq2p_process_Y(Y)
peakPValueX = sq2p_process_X(peakPValueX)
peakPValueY = sq2p_process_Y(peakPValueY)
peakBinaryX = sq2p_process_X(peakBinaryX)
peakBinaryY = sq2p_process_Y(peakBinaryY)    

In [4]:
class DataNormalizer(object):
    def __init__(self, mode):
        self.b = None
        self.W = None
        self.mode = mode
        if mode not in ['ZCA', 'Z', '01', 'identity']:
            raise ValueError("mode=%s must be 'ZCA', 'Z', '01', or 'identity'" % mode)


    def fit(self, X_orig):
        """
        Learns scaling parameters on the X_orig dataset. Does not modify X_orig.
        """        
        if len(X_orig.shape) != 2 and len(X_orig.shape) != 3:
            raise ValueError("X must be either a 3-tensor of shape num_examples x seq_length x \
                               num_input_marks, or a 2-tensor of shape num_examples x num_input_marks")
        if self.mode == 'identity':
            return None        

        X = np.copy(X_orig)
        num_input_marks = X.shape[-1]

        # If X is a 3-tensor, reshape X such that it is a 2-tensor of shape 
        # (num_examples * seq_length) x num_input_marks. 
        if len(X.shape) == 3:    
            X = np.reshape(X, (-1, num_input_marks))
        
        self.b = np.mean(X, axis=0) 

        X -= self.b

        if self.mode == 'ZCA':
            sigma = np.dot(X.T, X) / X.shape[0]
            U, S, V = np.linalg.svd(sigma)
            self.W = np.dot(
                np.dot(U, np.diag(1 / np.sqrt(S + 1e-5))),
                U.T)
        elif self.mode == 'Z':
            self.W = np.empty(num_input_marks)
            for idx in range(num_input_marks):
                self.W[idx] = np.std(X[:, idx])
        elif self.mode == '01':
            self.W = np.empty(num_input_marks)
            for idx in range(num_input_marks):
                self.W[idx] = np.max(np.abs(X[:, idx]))

        return None            


    def transform(self, X):
        if len(X.shape) != 2 and len(X.shape) != 3:
            raise ValueError("X must be either a 3-tensor of shape num_examples x seq_length x \
                               num_input_marks, or a 2-tensor of shape num_examples x num_input_marks")

        if self.mode == 'identity':
            return X
            
        assert self.b is not None
        assert self.W is not None

        num_input_marks = X.shape[-1]
        orig_shape = X.shape

        if self.mode == 'ZCA':            
            X = np.reshape(X, (-1, num_input_marks))
            if self.W.shape[1] != X.shape[1]:
                raise ValueError("When doing a ZCA transform, X and W must have the same number of columns.")
            X = np.dot(
                X - self.b,
                self.W.T)
            X = np.reshape(X, orig_shape)
        elif self.mode in ['Z', '01']:
            if (len(self.b) != num_input_marks) or (len(self.W) != num_input_marks):
                print("X.shape: ", X.shape)
                print("b.shape: ", self.b.shape)
                print("W.shape: ", self.W.shape)
                raise ValueError("The shapes of X, b, and W must all share the same last dimension.")                
            for idx in range(num_input_marks):
                X[..., idx] = (X[..., idx] - self.b[idx]) / self.W[idx]

        return X

In [5]:
scale_input = "01"
normalizer = DataNormalizer(scale_input)
normalizer.fit(X)
X = normalizer.transform(X)

In [6]:
import keras
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.convolutional import UpSampling1D

NUM_FILTERS = 6
FILTER_LEN  = 51
def Seq2Point_model(isBinaryModel):
    num_filters = NUM_FILTERS
    filter_length = FILTER_LEN
    seq_length = SEQ_LENGTH
    num_output_marks = len(OUTPUT_MARKS)
    num_input_marks = len(INPUT_MARKS)
    
    model = Sequential()
    # border_mode='same' makes the length of the output 
    # the same size as the length of the input
    # by adding just the right amount of zero padding to each side.
    model.add(
        Convolution1D(                    
            num_filters, 
            filter_length,
            input_dim=num_input_marks,
            init='uniform', 
            border_mode='same')) 

    model.add(Activation('relu'))

    # See below for documentation on border_mode='valid'
    # We are essentially replicating the "dense" layer here, but with a convolutional layer
    # so that later we can do genome-wide prediction.
    model.add(
        Convolution1D(
            num_output_marks, # output_dim --> always 1                    
            seq_length,
            init='uniform',
            border_mode='valid'))

    if isBinaryModel: # one of these values is True and other is False
        model.add(Activation('sigmoid')) # return 0 ~ 1. It means binary classification 
    else: 
        model.add(Activation('relu')) # return some value. It means signal regression

    
    print(model.summary())


    return model

Using TensorFlow backend.


In [7]:
model_sig = Seq2Point_model(isBinaryModel=False)
model_bin = Seq2Point_model(isBinaryModel=True)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


/home/ncp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(6, 51, input_shape=(None, 6), padding="same", kernel_initializer="uniform")`
/home/ncp/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(1, 1001, padding="valid", kernel_initializer="uniform")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 6)           1842      
_________________________________________________________________
activation_1 (Activation)    (None, None, 6)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 1)           6007      
_________________________________________________________________
activation_2 (Activation)    (None, None, 1)           0         
Total params: 7,849
Trainable params: 7,849
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, None, 6)           1842      
_________________________________________________________________
act

In [8]:
seq2point_weight_sig  = 's2q-sig'
sig_checkpointer = ModelCheckpoint(
    filepath=os.path.join('.', '%s-weights.hdf5'%seq2point_weight_sig), 
    verbose=1, 
    save_best_only=True)

seq2point_weight_bin = 's2q-bin'
bin_checkpointer = ModelCheckpoint(
    filepath=os.path.join('.', '%s-weights.hdf5'%seq2point_weight_bin), 
    verbose=1, 
    save_best_only=True)

sig_earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=0)
bin_earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=0)

In [9]:
NB_EPOCH          = 100
VALID_SPLIT       = 0.1
BATCH_SIZE        = 100
# complie seq2point model
model_sig.compile(loss='MSE', optimizer="Adam")
model_bin.compile(loss='binary_crossentropy', optimizer="Adam")

In [10]:
# train seq2point model
print('Seq2Point sig model fit start ')
hist_sig = model_sig.fit(X, Y, 
                         callbacks=[sig_checkpointer, sig_earlystopper],
                         nb_epoch=NB_EPOCH,
                         validation_split=VALID_SPLIT,
                         batch_size=BATCH_SIZE)
print('----------------------------------------------------------------')

Seq2Point sig model fit start 


/home/ncp/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 2s 219us/step - loss: 0.7919 - val_loss: 0.3327

Epoch 00001: val_loss improved from inf to 0.33273, saving model to ./s2q-sig-weights.hdf5
Epoch 2/100
9000/9000 [==============================] - 1s 64us/step - loss: 0.2698 - val_loss: 0.2892

Epoch 00002: val_loss improved from 0.33273 to 0.28924, saving model to ./s2q-sig-weights.hdf5
Epoch 3/100
9000/9000 [==============================] - 1s 63us/step - loss: 0.2527 - val_loss: 0.2849

Epoch 00003: val_loss improved from 0.28924 to 0.28492, saving model to ./s2q-sig-weights.hdf5
Epoch 4/100
9000/9000 [==============================] - 1s 63us/step - loss: 0.2486 - val_loss: 0.3286

Epoch 00004: val_loss did not improve from 0.28492
Epoch 5/100
9000/9000 [==============================] - 1s 63us/step - loss: 0.2395 - val_loss: 0.2846

Epoch 00005: val_loss improved from 0.28492 to 0.28463, saving model to ./s2q-sig-weights.hdf5

In [11]:
print('Seq2Point bin model fit start ')
hist_bin = model_bin.fit(X, peakBinaryY, 
                         callbacks=[bin_checkpointer, bin_earlystopper],
                         nb_epoch=NB_EPOCH,
                         validation_split=VALID_SPLIT,
                         batch_size=BATCH_SIZE)
print('----------------------------------------------------------------')

Seq2Point bin model fit start 


/home/ncp/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 1s 90us/step - loss: 0.2292 - val_loss: 0.1679

Epoch 00001: val_loss improved from inf to 0.16794, saving model to ./s2q-bin-weights.hdf5
Epoch 2/100
9000/9000 [==============================] - 1s 64us/step - loss: 0.1373 - val_loss: 0.1782

Epoch 00002: val_loss did not improve from 0.16794
Epoch 3/100
9000/9000 [==============================] - 1s 66us/step - loss: 0.1345 - val_loss: 0.1746

Epoch 00003: val_loss did not improve from 0.16794
Epoch 4/100
9000/9000 [==============================] - 1s 64us/step - loss: 0.1287 - val_loss: 0.1760

Epoch 00004: val_loss did not improve from 0.16794
Epoch 5/100
9000/9000 [==============================] - 1s 64us/step - loss: 0.1262 - val_loss: 0.1645

Epoch 00005: val_loss improved from 0.16794 to 0.16448, saving model to ./s2q-bin-weights.hdf5
Epoch 6/100
9000/9000 [==============================] - 1s 69us/step - loss: 0.1234 - v